<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/tensorflow_Kobert_%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.4.2

     |████████████████████████████████| 2.0 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 65.0 MB/s 
     |████████████████████████████████| 895 kB 67.8 MB/s 


transformers라는 패키지는 허깅페이스라는 스타트업에서 만들었습니다.

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [3]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import urllib.request
import logging
import tensorflow_addons as tfa

In [5]:
logging.basicConfig(level=logging.ERROR)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/202109_nlp_renew/train_0902.csv')

#라벨 데이터 category 정수로 변환

data_colname = 'contents'
label_colname = 'category'

encoder = LabelEncoder()
encoder.fit(dataset[label_colname])
dataset[label_colname] = encoder.transform(dataset[label_colname])


#이어서 contents 전처리(정제)
dataset[data_colname] = dataset[data_colname].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("무단전재 및 재배포 금지"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("©"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("ⓒ"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("저작권자"," ",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
dataset[data_colname] = dataset[data_colname].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
dataset[data_colname] = dataset[data_colname].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
dataset[data_colname] = dataset[data_colname].str.replace('\"', "",regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
dataset[data_colname] = dataset[data_colname].str.replace("\n"," ")
dataset[data_colname] = dataset[data_colname].str.replace("\r"," ")
dataset[data_colname] = dataset[data_colname].str.replace("\t"," ")
dataset[data_colname] = dataset[data_colname].str.replace( "\’" , "", regex=True)
dataset[data_colname] = dataset[data_colname].str.replace("[ ]{2,}"," ",regex=True)

dataset.head()

,id,contents,category
0,NWRW2000000001.1,조국 330개 문건 을 한 데 대해선 정부의 각종 기밀이 언론에 유출됐을 때는 동...,8
1,NWRW2000000001.2,김 프로 그거 알아? 캐디한테 방향 봐달라 하면 이제 반칙이야 모든 스포츠 중 가...,6
2,NWRW2000000001.3,"임종석, 정부의 KT&G 김 수사관이 자신의 비위 행위를 숨기고자 희대의 농간을 ...",8
3,NWRW2000000001.4,'를 요구했다. 주요 선진국에서 젊은 리더가 등장할 수 있었던 것은 정부·정당들이...,8
4,NWRW2000000001.5,"7월 光州 세계수영선수권, 11월 야구 국가대항전… 축구 말고도 볼 게 많네 지난...",6


In [49]:
# # 앞에서 뒤에서 자르기
# for idx, x in enumerate(dataset['contents']):
#   if(len(x) > 510):
#     head = x[:128]
#     tail = x[-382:]
#     x = head + ' ' + tail

In [10]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'IT/과학',
 1: '경제',
 2: '문화',
 3: '미용/건강',
 4: '사회',
 5: '생활',
 6: '스포츠',
 7: '연예',
 8: '정치'}

In [11]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [12]:
print('훈련용 데이터 개수 :',len(train)) # 훈련용 데이터 개수 출력

훈련용 데이터 개수 : 90615


In [13]:
print('테스트용 데이터 개수 :',len(test)) # 테스트용 데이터 개수 출력

테스트용 데이터 개수 : 22654


In [14]:
train.reset_index(drop=True, inplace=True) #중요!!!!
test.reset_index(drop=True, inplace=True) #중요!!!!

In [15]:
train[:5] # 상위 5개 출력

,id,contents,category
0,NIRW2000000002.978,"현대차, 광주형 일자리 투자 확정 경형 SUV로 新시장 개척 현대자동차가 국내 최...",4
1,NWRW2000000002.3853,회견에서도 최 부상은 귀국길에 김 위원장이 ‘대체 무슨 이유로 우리가 다시 이런 ...,4
2,NIRW2000000001.24636,국악과 함께하는 경복궁 봄 나들이 어떠세요? 전통공연예술진흥재단 에서 소원을 이루...,2
3,NWRW2000000002.24759,"검찰, 삼성바이오 분식회계 ‘회계법인부터 겨눈다 삼성바이오로직스 는 “삼정과 삼일...",4
4,NWRW2000000002.2558,외제차·여행 비용에 실종된 반려견 전단지값까지…‘알뜰하게 받아챙긴 법원공무원 대법...,4


In [16]:
test[:5] # 상위 5개 출력

,id,contents,category
0,NWRW2000000003.2116,트럼프 국가 비상사태 vs 美언론 트럼프 비상사태 멕시코 국경장벽 건설비 확보를 ...,4
1,NIRW2000000001.18591,"광주세계수영선수권대회, 수구경기 진출팀 최종 확정 90여 일 앞으로 다가온 광주세...",6
2,NIRW2000000001.13136,실에서 회견을 갖고 '플라즈마 열분해 가스에너지화 시범사업' 추진에 반대한다고 밝...,4
3,NWRW2000000003.22684,오지·낙도서 '살고 싶은 곳‘ 거듭난 전남 섬… 주민 소득도 껑충 ‘오지·낙도로 ...,5
4,NWRW2000000003.22690,"롯데푸드 중화권에 분유 수출 확대 추진…대만, 홍콩, 마카오에 우선 판매 롯데푸드...",5


Hugginface라는 스타트업의 transformers라는 패키지를 설치하시면  
모델 사용할 때는 항상 model.from_pretrained()

In [17]:
# 모델 로드
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Koelectra 깃허브 : https://github.com/monologg/KoELECTRA

모델의 출력을 체크해봅시다.

In [18]:
max_seq_len = 128

In [19]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [20]:
outputs

TFBaseModelOutputWithPooling([('last_hidden_state',
                               <KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model')>),
                              ('pooler_output',
                               <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)])

출력이 2개입니다.

outputs[0]은 모든 시점의 BERT의 최종 벡터. 문장 길이가 64일 경우에는 64 x 768  
outputs[1]은 CLS 시점의 BERT의 벡터

In [19]:
outputs[0]

<KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [20]:
outputs[1]

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>

In [21]:
# 토크나이저 코드 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py", filename="tokenization_kobert.py")

('tokenization_kobert.py', <http.client.HTTPMessage at 0x7f81e8db6850>)

In [22]:
from tokenization_kobert import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [68]:
text = """
기사 제도가 발전하면서 기독교 세계에서는 교회를 존경하고 영주와 군대의 상관에게 충성하며 자기 명예를 지키는 이라는 이상적인 기사상(騎士像)이 널리 퍼졌다. 이런 이상에 가까운 기사들이 나타난 것은 11세기말부터 유럽 기독교 세계의 기사들이 교회를 보호한다는 공동 대의 아래 모였던 십자군전쟁 때였다. 십자군전쟁 때 최초의 기사단들, 즉 예루살렘의 구호기사단과 성전(聖殿)기사단이 생겼고 이보다 좀 뒤에는 나병환자 병원을 지키는 특별임무를 맡은 성(聖) 라자루스 기사단이 조직되었다. 이 기사단들은 그 목적이나 형태로 볼 때 종교적·국제적인 성격을 띠고 있었다. 기사들은 독신생활을 했고 위계질서를 갖추고 있어 기사단은 교회의 직제(職制)를 그대로 본떠 기사단장과 관구장(官區長)을 비롯해 수도원장·분단장·일반기사로 구성되었다. 그러나 기사단의 기사 수와 재물이 늘어남에 따라 얼마 안가서 그들은 종교적 목적은 제쳐두고 정치활동을 벌이게 되었다.

이와 동시에 국가적인 문제에 더 비중을 둔 십자군 기사단들이 나타났다. 스페인의 카스티야 왕국에서는 이슬람교도와 싸우고 순례자를 보호한다는 목적 아래 1156~71년에 칼라트라바·알칸타라·산티아고(성 야코부스) 등의 기사단이 각각 생겨났고 같은 시기에 포르투갈에서는 아비스 기사단이 조직되었다. 아라곤 왕국의 몬테사 기사단(1317)과 포르투갈의 그리스도 기사단은 성전 기사단이 해체된 뒤에야 비로소 생겨났고 독일 출신 기사들도 대규모 기사단을 조직했는데(→ 튜턴 기사단), 이런 국가편향적인 십자군 기사단들은 다른 국제적인 규모의 기사단들처럼 세속적인 입지를 넓히는 데 치중했다. 그러나 이들이 유럽에서 벌인 활동은 팔레스타인에서 벌인 국제적인 활약 못지 않게 오랫동안 해외의 기사들과 심지어는 기사가 아닌 사람들의 마음까지도 사로잡았다.

11세기말~13세기 중반에는 봉건제와 기사제도의 관계가 변하기 시작했다. 봉건영주들은 기사들에게 봉토를 주고 보통 매년 40일 동안 군역의무를 지웠다. 이것은 왕의 영토를 지키고 봉사의무를 수행하기에는 충분했다. 그러나 십자군전쟁을 비롯해 백년전쟁 때 벌인 장기(長期) 침공작전처럼 잦은 장거리 해외 원정에는 맞지 않아 국왕이 토지보유자들에게 기사가 되라고 강요하는 일이 늘어났다. 또한 돈을 받고 싸우는 용병(庸兵)들이 군대에 점점 많아졌으므로 과거 군대의 주류를 이루던 기사들은 소수파가 되어 일종의 장교계층과 비슷하게 되었다.

점차 십자군운동이 시들해지고, 기사들로 이루어진 기병대가 보병과 궁수(弓手)부대에게 참담한 패배를 당하는 일이 생기는 한편 대포의 발달과 중앙집권제를 통한 왕권강화로 14~15세기에 걸쳐 전통적인 기사 제도는 무너졌다. 게다가 16세기에 들어와서는 군사적인 의미를 잃고 국왕이 마음 내킬 때 수여하는 명예 지위로 전락했다. 군주의 측근인 고위 귀족들 사이에서는 이같은 칭호를 갖는 것이 유행했다.

중세말부터 종교와 관계없는 세속적인 기사작위들이 많이 생겨 이들에게 훈장(勳章)을 주게 되었다. 이런 작위는 귀족이나 행정관리, 각종 직업과 예술분야에서 훌륭한 업적을 이룩한 사람들에게 수여했다. 오늘날 이같은 작위는 영국의 경우 국왕과 작위를 받을 사람 모두 개인자격으로 참여하는 의식을 통해 받는 명예 칭호에 불과하다. 오늘날 작위를 받는 사람이 국왕 앞에 무릎을 꿇으면 국왕은 용검(龍劍：영국 국가의식 때 국왕 앞에서 받드는 칼)을 칼집에서 뽑아 수상자의 오른쪽 어깨와 왼쪽 어깨에 차례로 가볍게 댄다. 수상자가 남자일 경우에는 이름에 '써'(Sir, 卿), 여자는 '데임'(Dame) 칭호를 붙인다.

기사는 사용하는 무기에 따라 호칭이 달랐다. 활을 사용하면 궁기사, 창을 사용하면 창기사라 불렸는데 총을 사용하면 용기병(dragoon)이라 불렀다. 총을 사용하는 기사가 용기사라 불리는 이유는, 용기병이 사용하는 총에 드래곤의 문양이 박혀져 있어 기병용 총을 '드래건'이라고 불렀기 때문이다. 그런데 엄밀히 말하면 용기병들은 이동에만 말을 이용하고 싸울때는 말에서 내려서 전투하는 하마보병에 가까웠다.
"""

In [69]:
head = text[:128]
tail = text[-382:]

In [70]:
tail

"작위를 받는 사람이 국왕 앞에 무릎을 꿇으면 국왕은 용검(龍劍：영국 국가의식 때 국왕 앞에서 받드는 칼)을 칼집에서 뽑아 수상자의 오른쪽 어깨와 왼쪽 어깨에 차례로 가볍게 댄다. 수상자가 남자일 경우에는 이름에 '써'(Sir, 卿), 여자는 '데임'(Dame) 칭호를 붙인다.\n\n기사는 사용하는 무기에 따라 호칭이 달랐다. 활을 사용하면 궁기사, 창을 사용하면 창기사라 불렸는데 총을 사용하면 용기병(dragoon)이라 불렀다. 총을 사용하는 기사가 용기사라 불리는 이유는, 용기병이 사용하는 총에 드래곤의 문양이 박혀져 있어 기병용 총을 '드래건'이라고 불렀기 때문이다. 그런데 엄밀히 말하면 용기병들은 이동에만 말을 이용하고 싸울때는 말에서 내려서 전투하는 하마보병에 가까웠다.\n"

In [72]:
text = head + ' ' + tail
print(len(text))

511


In [73]:
len(tokenizer.encode(text))

287

In [74]:
print(tokenizer.tokenize(text))

['▁기사', '▁제도', '가', '▁발전', '하면서', '▁기', '독', '교', '▁세계', '에서는', '▁교', '회를', '▁존', '경', '하고', '▁영', '주', '와', '▁군', '대의', '▁상', '관', '에게', '▁충', '성', '하며', '▁자기', '▁명예', '를', '▁지키', '는', '▁', '이라는', '▁이상', '적인', '▁기사', '상', '(', '騎士像', ')', '이', '▁', '널', '리', '▁퍼', '졌다', '.', '▁이런', '▁이상', '에', '▁가까운', '▁기사', '들이', '▁나타난', '▁것은', '▁11', '세', '기', '말', '부터', '▁유럽', '▁기', '독', '교', '▁세', '▁작', '위를', '▁받는', '▁사람이', '▁국', '왕', '▁앞에', '▁무릎', '을', '▁', '꿇', '으면', '▁국', '왕', '은', '▁용', '검', '(', '龍劍', ':', '영', '국', '▁국가', '의', '식', '▁때', '▁국', '왕', '▁앞에서', '▁받', '드는', '▁칼', ')', '을', '▁칼', '집', '에서', '▁뽑아', '▁수상', '자의', '▁', '오른쪽', '▁어깨', '와', '▁왼쪽', '▁어깨', '에', '▁차례', '로', '▁', '가', '볍', '게', '▁', '댄', '다', '.', '▁수상', '자가', '▁남자', '일', '▁경우', '에는', '▁이름', '에', "▁'", '써', "'(", 'S', 'ir', ',', '▁', '卿', '),', '▁여자', '는', "▁'", '데', '임', "'(", 'D', 'am', 'e', ')', '▁', '칭', '호', '를', '▁붙', '인', '다', '.', '▁기사', '는', '▁사용하는', '▁무기', '에', '▁따라', '▁호', '칭', '이', '▁달', '랐', '다', '.', '▁', '활', '

In [75]:
for elem in tokenizer.encode(text):
  print(tokenizer.decode(elem))

[CLS]
기사
제도
가
발전
하면서
기
독
교
세계
에서는
교
회를
존
경
하고
영
주
와
군
대의
상
관
에게
충
성
하며
자기
명예
를
지키
는

이라는
이상
적인
기사
상
(
[UNK]
)
이

널
리
퍼
졌다
.
이런
이상
에
가까운
기사
들이
나타난
것은
11
세
기
말
부터
유럽
기
독
교
세
작
위를
받는
사람이
국
왕
앞에
무릎
을

꿇
으면
국
왕
은
용
검
(
[UNK]
:
영
국
국가
의
식
때
국
왕
앞에서
받
드는
칼
)
을
칼
집
에서
뽑아
수상
자의

오른쪽
어깨
와
왼쪽
어깨
에
차례
로

가
볍
게

댄
다
.
수상
자가
남자
일
경우
에는
이름
에
'
써
'(
S
ir
,

[UNK]
),
여자
는
'
데
임
'(
D
am
e
)

칭
호
를
붙
인
다
.
기사
는
사용하는
무기
에
따라
호
칭
이
달
랐
다
.

활
을
사용
하면

궁
기사
,
창
을
사용
하면
창
기사
라
불
렸
는데
총
을
사용
하면
용
기
병
(
d
ra
go
on
)
이라
불
렀다
.
총
을
사용하는
기사
가
용
기사
라
불리는
이유는
,
용
기
병
이
사용하는
총
에

드래곤
의
문
양
이
박
혀
져
있어
기
병
용
총
을
'
드
래
건
'
이라고
불
렀
기
때문이다
.
그런데
엄
밀
히
말
하면
용
기
병
들은
이동
에만
말을
이용
하고
싸
울
때
는
말
에서
내려
서
전투
하는
하
마
보
병
에

가
까
웠다
.
[SEP]


In [ ]:
for elem in tokenizer.encode("happy birthday~!"):
  print(tokenizer.decode(elem))

[CLS]

ha
p
p
y

b
ir
th
d
ay
~
!
[SEP]


In [80]:
print(tokenizer.decode(2))

[CLS]


In [77]:
print(tokenizer.decode(3))

[SEP]


In [ ]:
max_seq_len=512

In [78]:
print(tokenizer.encode(text, max_length=max_seq_len, pad_to_max_length=True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 1283, 4137, 5330, 2249, 7812, 1258, 5865, 5488, 2802, 6904, 1103, 7955, 4192, 5424, 7788, 3376, 7276, 6983, 1165, 5818, 2658, 5474, 6897, 4588, 6573, 7810, 3890, 2037, 6116, 4344, 5760, 517, 7103, 3704, 7206, 1283, 6527, 18, 0, 40, 7096, 517, 5696, 6122, 4826, 7250, 54, 3672, 3704, 6896, 734, 1283, 5940, 1400, 910, 538, 6579, 5561, 6160, 6410, 3581, 1258, 5865, 5488, 2801, 3930, 7049, 2224, 2589, 1132, 6992, 3190, 2106, 7088, 517, 5631, 7083, 1132, 6992, 7086, 3494, 5392, 18, 0, 249, 6951, 5503, 1133, 7095, 6730, 1844, 1132, 6992, 3191, 2220, 5921, 4642, 40, 7088, 4642, 7354, 6903, 2569, 2884, 7167, 517, 6967, 3221, 6983, 3479, 3221, 6896, 4407, 6079, 517, 5330, 6359, 5400, 517, 5831, 5782, 54, 2884, 7148, 1423, 7126, 968, 6900, 3687, 6896, 520, 6761, 16, 343, 411, 46, 517, 0, 42, 3318, 5760, 520, 5850, 7136, 16, 278, 373, 389, 40, 517, 7494, 7925, 6116, 2503, 7119, 5782, 54, 1283, 5760, 2615, 2097, 6896, 1835, 5090, 7494, 7096, 1597, 6021, 5782, 54, 517, 7948, 7088, 2613, 7811, 51

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



토큰 인풋의 예를 들면 다음과 같습니다.  
문장을 토크나이징 하면 "전율을 일으키는 영화. 다시 보고싶은 영화"가
"'전', '##율', '##을', '일', '##으', '##키는', '영화', '.', '다시', '보고', '##싶', '##은', '영화'" 로 토크나이징이 됩니다.  
이거를 버트 인풋에 들어갈 숫자로 바꿔주면,  
["2, 4012, 7071, 3815, 5760, 3394, 54, 1574, 2358, 6751, 7086, 3394, 3"]  
로 바뀌게 됩니다. 여기 나오는 숫자들이 버트 인풋에 들어가는 토큰 인풋입니다.
버트 모형에 들어가는 인풋은 사실 일정한 길이를 가져야 합니다.(본 예제에서는 64)
따라서 남는 부분은 1로 채워지게 됩니다(패딩)

In [81]:
# 세그멘트 인풋
print([0]*512)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

세그멘트 인풋은 문장이 앞문장인지 뒷문장인지 구분해주는 역할을 하는데요 본 문장에서는 문장 하나만 인풋으로 들어가기 때문에 0만 들어가게 되고, 문장 길이만큼의 0이 인풋으로 들어가게 됩니다.

In [82]:
# 마스크 인풋
valid_num = len(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

숫자 2 : [CLS]  
숫자 3 : [SEP]


어텐션 마스크 인풋은 토큰 인풋에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 두게 됩니다.  

종합하면,
버트의 인풋은 토큰, 세그먼트, 마스크로 이루어져 있습니다.
"전율을 일으키는 영화. 다시 보고싶은 영화" 라는 문장을 가지고 예를 들면,  

토큰 인풋 : 
[2, 4012, 7071, 3815, 5760, 3394, 54, 1574, 2358, 6751, 7086, 3394, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

세그먼트 인풋 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

어텐션 마스크 인풋 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  

네이버 영화 평가 문장들을 버트 인풋으로 바꿔보도록 하겠습니다.
문장이 토큰 인풋, 세그먼트 인풋, 마스크 인풋으로 변환 됩니다.
huggingface에서는 순서가 [토큰 인풋, 마스크 인풋, 세그먼트 인풋] 입니다.

In [23]:
from tensorflow.keras.utils import to_categorical
NUM_CLASS = 9

def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
       
        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id는 세그먼트 임베딩을 위한 것으로 이번 예제는 문장이 1개이므로 전부 0으로 통일.
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label = to_categorical(label, num_classes=NUM_CLASS, dtype=np.int32)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [24]:
max_seq_len = 128

In [25]:
train_X, train_y = convert_examples_to_features(train['contents'], train['category'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/90615 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 90615/90615 [02:32<00:00, 595.66it/s]


In [26]:
test_X, test_y = convert_examples_to_features(test['contents'], test['category'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/22654 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 22654/22654 [00:39<00:00, 569.53it/s]


In [27]:
# 최대 길이: 128
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [   2 5056   46 1098 7921 3819 4768 5134  953 7921  687  356  357 6079
  517 5282 6716  853 5054 5330 1138 4539 7095 1485 2669 7921 3819 2046
 6079 1349 5760  520 5481 7276 7921 3819   15 2609 6896 1381 5407 1046
 2253   54 5056 5760  596 7126 1098 5483 4220 7095 3014 3893 2714 4984
 7170 6338 6896 4241 7095 3803 7023 4435 7786 2261   54 1099 6983 5056
 1826 2714 4984 7170 6338 7088 2772 7848 3519 5889  520 5481 7276 7921
 3819   15 6896 1046 7203 4435 3629 6116 2264  913   54 4768 5077 6896
 1838 3019 6338 7086 3902 5550 3211  553  223 6861 1815 4512  517  209
 6861 1180 2772 5900   54 1098 5330 4527 4241 6079 3902 5550 7095  553
   94    3]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
세그먼트 인코딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [28]:
train_X[0].shape, train_X[1].shape, train_X[2].shape, train_y.shape

((90615, 128), (90615, 128), (90615, 128), (90615, 9))

TPU 사용법 : https://wikidocs.net/119990

In [29]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.72.88.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.72.88.106:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [30]:
num_epochs = 5
num_batch = 16

In [31]:
warmup_ratio = 0.1

t_total = len(train) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [32]:
LR = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=2e-5, first_decay_steps=warmup_step)

In [33]:
initializer_range = 0.2

In [34]:
def create_model():
  model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

  input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
  attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
  token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

  outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])
  pooled_output = outputs[1]

  optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
  
  pooled_output = tf.keras.layers.Dropout(0.5)(pooled_output)
  prediction = tf.keras.layers.Dense(9, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=initializer_range))(pooled_output)
  cls_model = tf.keras.Model([input_ids_layer, attention_masks_layer, token_type_ids_layer], prediction)
  cls_model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
  cls_model.summary()

  return cls_model

In [35]:
from google.colab import auth
auth.authenticate_user()

In [38]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
####
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('gs://cateto/best_model', monitor='accuracy', mode='max', save_weights_only=True, verbose=1, save_best_only=True)
####
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  cls_model = create_model()
  cls_model.fit(train_X, train_y, callbacks=[es, mc], epochs=num_epochs, batch_size=num_batch, validation_data=(test_X, test_y))

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 9) dtype=int32>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 9) dtype=int32>]


5664/5664 [==============================] - ETA: 0s - loss: 0.6324 - accuracy: 0.7961

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_3:0' shape=(None, 9) dtype=int32>]


5664/5664 [==============================] - 466s 63ms/step - loss: 0.6324 - accuracy: 0.7961 - val_loss: 0.4676 - val_accuracy: 0.8435

Epoch 00001: accuracy improved from -inf to 0.79610, saving model to gs://cateto/best_model
Epoch 2/5
5664/5664 [==============================] - 355s 61ms/step - loss: 0.4218 - accuracy: 0.8595 - val_loss: 0.4632 - val_accuracy: 0.8437

Epoch 00002: accuracy improved from 0.79610 to 0.85954, saving model to gs://cateto/best_model
Epoch 3/5
5664/5664 [==============================] - 357s 61ms/step - loss: 0.2964 - accuracy: 0.9012 - val_loss: 0.4605 - val_accuracy: 0.8575

Epoch 00003: accuracy improved from 0.85954 to 0.90123, saving model to gs://cateto/best_model
Epoch 4/5
5664/5664 [==============================] - 352s 60ms/step - loss: 0.1896 - accuracy: 0.9387 - val_loss: 0.5172 - val_accuracy: 0.8586

Epoch 00004: accuracy improved from 0.90123 to 0.93865, saving model to gs://cateto/best_model
Epoch 5/5
5664/5664 [========================

In [39]:
results = cls_model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

23/23 [==============================] - 19s 489ms/step - loss: 0.6553 - accuracy: 0.8559
test loss, test acc:  [0.6552583575248718, 0.8559194803237915]


In [40]:
def sentence_prediction(example):
    global tokenizer

    input_ids, attention_masks, token_type_ids = [], [], []

    input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
       
    # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
    # token_type_id는 세그먼트 임베딩을 위한 것으로 이번 예제는 문장이 1개이므로 전부 0으로 통일.
    token_type_id = [0] * max_seq_len
  
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

    input_ids = np.array(input_ids)
    attention_masks = np.array(attention_masks)
    token_type_ids = np.array(token_type_ids)
    return [input_ids, attention_masks, token_type_ids]

def evaluation_predict(sentence):
    data_x = sentence_prediction(sentence)
    predict = cls_model.predict(data_x)

    print('예측 결과 수치', predict)
    print(predict)
    predict_answer = np.argmax(predict[0])
    predict_value = predict[0][predict_answer]
    
    if predict_answer == 0:
      print("(IT/과학 확률 : %.2f) IT/과학 뉴스입니다." % (1-predict_value))
      return 0
    elif predict_answer == 1:
      print("(경제 확률 : %.2f) 경제 뉴스입니다." % predict_value)
      return 1
    elif predict_answer == 2:
      print("(문화 확률 : %.2f) 문화 뉴스입니다." % predict_value)
      return 2
    elif predict_answer == 3:
      print("(미용/건강 확률 : %.2f) 미용/건강 뉴스입니다." % predict_value)
      return 3
    elif predict_answer == 4:
      print("(사회 확률 : %.2f) 사회 뉴스입니다." % predict_value)
      return 4
    elif predict_answer == 5:
      print("(생활 확률 : %.2f) 생활 뉴스입니다." % predict_value)
      return 5
    elif predict_answer == 6:
      print("(스포츠 확률 : %.2f) 스포츠 뉴스입니다." % predict_value)
      return 6
    elif predict_answer == 7:
      print("(연예 확률 : %.2f) 연예 뉴스입니다." % predict_value)
      return 7
    elif predict_answer == 8:
      print("(정치 확률 : %.2f) 정치 뉴스입니다." % predict_value)
      return 8

In [41]:
text = """
진중권 전 동양대 교수는 이준석 국민의힘 대표에게 국민권익위원회가 발표한 부동산 거래내역 전수조사 결과 국민의힘 소속 의원들의 엄정한 조치를 요구했습니다. 이를 잘 해결할 경우 ‘리더십 리스크’를 해결할 수 있을 것이라 내다봤습니다.
진 전 교수는 지난 23일 CBS라디오 ‘한판 승부’에 출연해 국민권익위 전수조사 결과 국민의힘 의원 12명에 대해 투기 의혹이 제기된 것에 대해 “대선에 큰 변수가 될 것 같지 않다”고 전망했습니다.
이어 “경찰수사, 검찰 수사에서 무혐의가 나올 것들이 상당히 많고 진짜 투기를 한 사람이 얼마나 되는지는 좀 더 지켜봐야” 한다며 “국민의힘에서 어떻게 처리할 것인가 더 큰 문제”라고 했습니다."""
evaluation_predict(text)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond_8/Identity_2:0' shape=(None, 128) dtype=int64>]


예측 결과 수치 [[9.1432346e-05 2.5908675e-02 5.0710285e-05 5.4441378e-05 5.1167244e-05
  1.6264745e-04 9.7335673e-05 1.3014692e-04 9.7345328e-01]]
[[9.1432346e-05 2.5908675e-02 5.0710285e-05 5.4441378e-05 5.1167244e-05
  1.6264745e-04 9.7335673e-05 1.3014692e-04 9.7345328e-01]]
(정치 확률 : 0.97) 정치 뉴스입니다.


8